In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import product, combinations
import matplotlib.pyplot as plt
import networkx as nx
from IPython.display import display, Math

In [3]:
# import code for representing profiles
from voting.profile_optimized import *
from voting.voting_methods_for_optimized import *
from voting.voting_methods_split_cycle import *
from generate_profiles import *

# display winners of a voting metho
def display_winners(vm, prof):
    print  "{} winners: {}".format(vm.__doc__, "{" + ", ".join([prof.cname(_) for _ in vm(prof)]) + "}")

# define some variables to simplify notation 
a=0
b=1
c=2
d=3
e=4
f=5

# candidate names are integers
cand_names_3 = [a,b,c]
cand_names_4 = [a,b,c,d]
cand_names_5 = [a,b,c,d,e]
cand_names_5 = [a,b,c,d,e,f]

# map candidate names to letters to match the notation in our papaer
cmap_abc = {a:"a", b:"b", c:"c"}
cmap_abcd = {a:"a", b:"b", c:"c", d:"d"}
cmap_abcde = {a:"a", b:"b", c:"c", d:"d", e:"e"}
cmap_abcdef = {a:"a", b:"b", c:"c", d:"d", e:"e", f:"f"}


In [4]:
# generate and display the margin graph
def display_margin_graph(prof): 
    # create marging graph
    mg = nx.DiGraph()
    mg.add_nodes_from([prof.cname(_) for _ in sorted(prof.candidates)])
    mg.add_weighted_edges_from([(prof.cname(c1),prof.cname(c2),prof.support(c1,c2) - prof.support(c2,c1))  
           for c1 in prof.candidates 
           for c2 in prof.candidates if c1 != c2 if prof.support(c1,c2) - prof.support(c2,c1) > 0])

    pos = nx.circular_layout(mg)
    nx.draw(mg, pos, font_size=20, node_color='blue', font_color='white',node_size=700, width=1, lw=1.5, with_labels=True)
    labels = nx.get_edge_attributes(mg,'weight')
    nx.draw_networkx_edge_labels(mg,pos,edge_labels=labels, label_pos=0.3)

    #nx.draw(mg)
    plt.title("Margin Graph")
    plt.show()


In [5]:
print splitcycle_faster.__doc__
print ranked_pairs.__doc__

Split Cycle (faster)
Ranked Pairs


In [6]:
num_profiles = 10000

number_of_voters = [101, 501, 1001] #6, 7, 9, 25, 30] #, 101, 1001]

phis = [1.0]
number_of_cands = [5] #5, 6, 7, 8]  


voting_situations = list(itertools.product(number_of_voters, number_of_cands))
rp_subset_sc_profs = list()
sc_subset_rp_profs = list()
just_diff = list()
num_diff = 0
num_rp_subset_sc = 0

voting_methods = [ ranked_pairs, splitcycle_faster]
for vs in voting_situations:
    print "\n voters: {}, cands: {}".format(vs[0], vs[1])
    num_voters = vs[0]
    num_cands = vs[1]
    
    cand_names, voter_names = generate_names_opt(num_cands, num_voters)

    cmap = generate_candidate_maps(cand_names)

    num_cands = len(cand_names)
    num_voters = len(voter_names)

    for phi in phis:
        profiles = [create_prof_opt_mallows(num_voters, cmap, phi) for _ in range(num_profiles)]

        print "done generating profiles...."
        for _prof in profiles:
            prof = ProfileOpt(_prof, cand_names, voter_names)
            rp_winners = ranked_pairs(prof)
            sc_winners = splitcycle_faster(prof)
            
            if rp_winners != sc_winners: 
                num_diff += 1
                if set(rp_winners).issubset(set(sc_winners)):
                    num_rp_subset_sc +=1
                    rp_subset_sc_profs.append(prof)
                elif set(sc_winners).issubset(set(rp_winners)):
                    print "FOUND ONE!!!"
                    prof.display_anon()
                    sc_subset_rp_profs.append(prof)
                else:
                    print "WOW: just different"
                    prof.display_anon()
                    print "RP ", rp_winners
                    print "SC ", sc_winners
                    just_diff.append(prof)
                    
            
print("done.")

print "num diff is ", num_diff
print "num rp subset sc ", num_rp_subset_sc


 voters: 101, cands: 5
done generating profiles....

 voters: 501, cands: 5
done generating profiles....

 voters: 1001, cands: 5
done generating profiles....
done.
num diff is  1029
num rp subset sc  1029
